In [1]:
from google.cloud import bigquery

In [2]:
client = bigquery.Client()
project_id = client.project

In [3]:
dataset_id = f"{project_id}.data_to_ai_workshop"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"

dataset = client.create_dataset(dataset, exists_ok=True)

In [4]:
uri = "gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv"
table_id = f"{project_id}.data_to_ai_workshop.weather_data"


In [5]:
job_config = bigquery.LoadJobConfig(
    source_format = bigquery.SourceFormat.CSV,
    skip_leading_rows = 1,
    autodetect = True
)

In [6]:
load_job = client.load_table_from_uri(uri, table_id, job_config = job_config)
load_job.result()

LoadJob<project=qwiklabs-gcp-01-a82571ac193f, location=US, id=4f71494c-c2a0-4903-9214-f473036a5943>

In [7]:
%%bigquery
SELECT *
FROM `data_to_ai_workshop.weather_data`
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,date,city,state,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent,weather_condition
0,2025-02-21,Atlanta,GA,55.7,5.0,0.12,29.80,50.4,Cloudy
1,2025-02-26,Atlanta,GA,75.2,10.4,0.03,29.58,49.9,Cloudy
2,2025-03-01,Atlanta,GA,51.7,4.7,0.08,29.74,49.9,Cloudy
3,2025-03-05,Atlanta,GA,74.4,5.1,0.02,29.92,50.4,Cloudy
4,2025-03-10,Atlanta,GA,59.5,9.6,0.09,29.67,57.2,Cloudy


In [8]:
%%bigquery
CREATE OR REPLACE MODEL `data_to_ai_workshop.gemini_model`
REMOTE WITH CONNECTION DEFAULT
OPTIONS(endpoint = 'gemini-2.0-flash-001');

Query is running:   0%|          |

""


In [9]:
%%bigquery
CREATE OR REPLACE TABLE `data_to_ai_workshop.weather_reports` AS
SELECT
  *,
  ml_generate_text_result['candidates'][0]['content']['parts'][0]['text'] AS ai_weather_report
FROM
  ML.GENERATE_TEXT(
    MODEL `data_to_ai_workshop.gemini_model`,
    (
      SELECT
        *,
        CONCAT(
          "Write a weather warning or report for ", city, ", ", state,
          " on ", CAST(date AS STRING), ". Conditions: ", weather_condition,
          ", Temp: ", CAST(temperature_f AS STRING), "F, Wind: ",
          CAST(wind_speed_mph AS STRING), "mph."
        ) AS prompt
      FROM
        `data_to_ai_workshop.weather_data`
    ),
    STRUCT(
      0.4 AS temperature,
      1024 AS max_output_tokens,
      0.8 AS top_p
    )
  );

Query is running:   0%|          |

""


In [10]:
%%bigquery
SELECT ai_weather_report
FROM `data_to_ai_workshop.weather_reports`
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,ai_weather_report
0,"""## Excessive Heat Warning for Atlanta, GA - M..."
1,"""## Atlanta, GA Weather Report - Friday, Febru..."
2,"""## Winter Weather Advisory for Boston, MA\n\n..."
3,"""## Miami-Dade County Weather Report - March 1..."
4,"""## Miami, FL - Weather Report - March 11, 202..."
